# Model Forecasting

## Requirements

In [2]:
#import necesary libraries
import pandas as pd

from utils import load

## Data Load

In [3]:
df, resample_df = load()

## ARIMA Update & Forecasting

In [ ]:
#for cycle to iterate over folder
    #if ends with .pkl
        #load model
        #update model
        #forecast
        #plot